<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 流式处理与批处理

In [ ]:
from videos.walkthroughs import walkthrough_14 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将学习如何流式处理模型响应，以及批处理多个聊天补全请求。

---

## 目标

完成这个 notebook 后，您将：

- 学习如何流式处理模型响应。
- 学习如何批处理模型响应。
- 比较批处理和单个提示词聊天补全的性能。

---

## 导入

先从 `langchain_nvidia_ai_endpoints` 导入 `ChatNVIDIA` 类，这将使我们能够与本地的 Llama 3.1 NIM 进行交互。

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

---

## 创建模型实例

In [3]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 验证

在继续新的用例之前，让我们先验证一下能否通过 LangChain 与本地模型进行交互。

In [ ]:
prompt = 'Where and when was NVIDIA founded?'
result = llm.invoke(prompt)

In [ ]:
print(result.content)

---

## 流式响应

作为 `invoke` 方法的替代，您可以使用 `stream` 方法分块接收模型响应。这样，您就不必等待整个响应生成，可以在输出生成的过程中看到结果。流式输出可以带来更好的用户体验，尤其是对于长响应或者是在用户界面应用中。

这是一个会生成更长响应的提示词。

In [4]:
prompt = 'Explain who you are in roughly 500 words.'

给定这个提示词，咱们来看看 `stream` 函数是怎么工作的。

In [5]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='')

I am an artificial intelligence model designed to assist and communicate with humans. I'm a type of computer program that uses natural language processing (NLP) and machine learning algorithms to understand and generate human-like text. My primary function is to provide information, answer questions, and engage in conversation to the best of my abilities.

I don't have a physical body or a personal identity in the classical sense. I exist solely as a digital entity, running on computer servers and responding to input from users like you. My "existence" is a product of complex software and data, designed to simulate conversation and provide helpful responses.

 a massive corpus of text, which I use to learn patterns, relationships, and context. This corpus is sourced from various places, including books, articles, research papers, and online content. I've been trained on a wide range of topics, from science and history to entertainment and culture.

 I use this training data to generate

LangChain 中的 `stream` 方法作为一个基础工具，能够在响应生成的过程中逐步显示结果。这会让用户觉得与 LLM 的交互更灵敏，提高了用户体验。

---

在后续的 notebooks 中，我们将导入这个辅助函数来帮助我们。

---

## 批处理响应

您还可以使用 `batch` 对一系列输入进行提示调用。调用 `batch` 会返回一个与输入顺序一致的响应列表。

当处理需要 LLM 以某种方式回应的一组数据时，`batch` 不仅方便使用，而且其方法设计为能同时处理多个提示，它会尽可能并行运行响应。这使得多请求的处理更高效，减少了为一系列提示词生成响应所需的总时间。通过批处理请求，您可以利用语言模型的计算能力同时处理多个输入，提高性能和吞吐量。

我们将通过使用这组关于州首府的提示词来演示批处理的功能和性能优势。

In [ ]:
state_capital_questions = [
    'What is the capital of California?',
    'What is the capital of Texas?',
    'What is the capital of New York?',
    'What is the capital of Florida?',
    'What is the capital of Illinois?',
    'What is the capital of Ohio?'
]

使用 `batch` 我们可以传入整个列表...

In [ ]:
capitals = llm.batch(state_capital_questions)

... 然后返回一个响应列表。

In [ ]:
len(capitals)

In [ ]:
for capital in capitals:
    print(capital.content)

需要注意的是，`batch` 并不是在与 LLM 进行多轮对话（这个话题我们将在课程后面详细讨论）。相反，它是每次都向一个新的 LLM 实例提出多个问题。

---

## 比较 batch 和 invoke 的性能

为了快速观察批处理可能带来的性能提升，我们在这里记录一次对 `batch` 的调用时间。注意 `Wall time`。

In [ ]:
%%time
llm.batch(state_capital_questions)

现在为了比较，我们遍历 `state_capital_questions` 列表，对每个项目调用 `invoke`。同样，注意 `Wall time` 并将其与上面批处理的结果进行比较。

In [ ]:
%%time
for cq in state_capital_questions:
    llm.invoke(cq)

---

## 练习：批处理以创建常见问题文档

在这个练习中，您将使用批处理来回应一系列与 LLM 相关的问题，以创建一个常见问题文档（在这个 notebook 中，文档指的就是我们打印到屏幕的内容）。

以下是一些与 LLM 相关的问题列表。

In [ ]:
faq_questions = [
    'What is a Large Language Model (LLM)?',
    'How do LLMs work?',
    'What are some common applications of LLMs?',
    'What is fine-tuning in the context of LLMs?',
    'How do LLMs handle context?',
    'What are some limitations of LLMs?',
    'How do LLMs generate text?',
    'What is the importance of prompt engineering in LLMs?',
    'How can LLMs be used in chatbots?',
    'What are some ethical considerations when using LLMs?'
]

您的任务是填充下面的 `faq_answers`，为每个问题提供一系列回应。使用 `batch` 方法来轻松完成这项工作。

成功完成后，您应该能够打印调用 `create_faq_document` 时返回的值，传入 `faq_questions` 和 `faq_answers`，从而得到一个关于上述所有 LLM 相关问题的常见问题文档。

In [ ]:
def create_faq_document(faq_questions, faq_answers):
    faq_document = ''
    for question, response in zip(faq_questions, faq_answers):
        faq_document += f'{question.upper()}\n\n'
        faq_document += f'{response.content}\n\n'
        faq_document += '-'*30 + '\n\n'

    return faq_document

如果您遇到困难，请查看下面的*参考答案*。

### 您的代码

In [ ]:
faq_answers = []

In [ ]:
# This should work after you successfully populate `faq_answers` with LLM responses.
print(create_faq_document(faq_questions, faq_answers))

### 参考答案

In [ ]:
faq_answers = llm.batch(faq_questions)

In [ ]:
def create_faq_document(faq_questions, faq_answers):
    faq_document = ''
    for question, response in zip(faq_questions, faq_answers):
        faq_document += f'{question.upper()}\n\n'
        faq_document += f'{response.content}\n\n'
        faq_document += '-'*30 + '\n\n'

    return faq_document

In [ ]:
print(create_faq_document(faq_questions, faq_answers))

---

## 总结

在这个 notebook 中，您学习了如何流式和批处理模型响应，并使用批量 LLM 调用生成一个有用的常见问题文档。

在下一个 notebook 中，您将开始更加专注于提示词的创建，进行迭代提示工程和特定提示词的开发。